# Read Data and import Required library's

In [1]:
from warnings import filterwarnings
filterwarnings("ignore")
import pandas as pd
A = pd.read_csv("F:download/Cars93.csv")

In [2]:
A.columns

Index(['id', 'Manufacturer', 'Model', 'Type', 'Min.Price', 'Price',
       'Max.Price', 'MPG.city', 'MPG.highway', 'AirBags', 'DriveTrain',
       'Cylinders', 'EngineSize', 'Horsepower', 'RPM', 'Rev.per.mile',
       'Man.trans.avail', 'Fuel.tank.capacity', 'Passengers', 'Length',
       'Wheelbase', 'Width', 'Turn.circle', 'Rear.seat.room', 'Luggage.room',
       'Weight', 'Origin', 'Make'],
      dtype='object')

 we cannot use those column which contain dot in the column name so we remove dot from column name

In [3]:
A.columns = ['id','Manufacturer', 'Model', 'Type', 'MinPrice', 'Price', 'MaxPrice',
       'MPGcity', 'MPGhighway', 'AirBags', 'DriveTrain', 'Cylinders',
       'EngineSize', 'Horsepower', 'RPM', 'Revpermile', 'Mantransavail',
       'Fueltankcapacity', 'Passengers', 'Length', 'Wheelbase', 'Width',
       'Turncircle', 'Rearseatroom', 'Luggageroom', 'Weight', 'Origin',
       'Make']

In [4]:
A.head()

,id,Manufacturer,Model,Type,MinPrice,Price,MaxPrice,MPGcity,MPGhighway,AirBags,...,Passengers,Length,Wheelbase,Width,Turncircle,Rearseatroom,Luggageroom,Weight,Origin,Make
0,1,Acura,Integra,Small,12.9,15.9,18.8,25,31,None,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,2,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,3,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,4,Audi,100,Midsize,30.8,37.7,44.6,19,26,NaN,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,5,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


In [5]:
A = A.drop(["id"],axis=1)

# Missing Data 

In [6]:
A.isna().sum()

Manufacturer         0
Model                0
Type                 0
MinPrice             0
Price                0
MaxPrice             0
MPGcity              0
MPGhighway           0
AirBags              4
DriveTrain           0
Cylinders            0
EngineSize           0
Horsepower           0
RPM                  0
Revpermile           0
Mantransavail        0
Fueltankcapacity     0
Passengers           0
Length               0
Wheelbase            0
Width                0
Turncircle           0
Rearseatroom         2
Luggageroom         11
Weight               0
Origin               0
Make                 0
dtype: int64

In [7]:
for i in A.columns:
    if(A[i].dtypes=="object"):
        x = A[i].mode()[0]                # add mode value for categorical values
        A[i] = A[i].fillna(x)
    else:
        x = A[i].mean()                   # add mean value for continous values
        A[i] = A[i].fillna(x)

In [8]:
A.isna().sum()

Manufacturer        0
Model               0
Type                0
MinPrice            0
Price               0
MaxPrice            0
MPGcity             0
MPGhighway          0
AirBags             0
DriveTrain          0
Cylinders           0
EngineSize          0
Horsepower          0
RPM                 0
Revpermile          0
Mantransavail       0
Fueltankcapacity    0
Passengers          0
Length              0
Wheelbase           0
Width               0
Turncircle          0
Rearseatroom        0
Luggageroom         0
Weight              0
Origin              0
Make                0
dtype: int64

# Define X and Y

In [9]:
X = A.drop(labels=["Type"],axis=1)
Y = A[["Type"]]

# Feature Selection usig Anova and chi-Squre

In [10]:
def Anova(df,cat,con):
    from pandas import DataFrame
    from statsmodels.api import OLS
    from statsmodels.formula.api import ols
    rel = con + " ~ " + cat
    model = ols(rel,df).fit()
    from statsmodels.stats.anova import anova_lm
    anova_result = anova_lm(model)
    Q = DataFrame(anova_result)
    a= Q["PR(>F)"][cat]
    return round(a,4)

In [11]:
imp_col = []
for i in X.columns:
    if(X[i].dtypes!="object"):
        x = Anova(A,"Type",i)
        if(x<0.05):
            imp_col.append(i)
            print("Type vs ",i,"--->",x)

Type vs  MinPrice ---> 0.0
Type vs  Price ---> 0.0
Type vs  MaxPrice ---> 0.0
Type vs  MPGcity ---> 0.0
Type vs  MPGhighway ---> 0.0
Type vs  EngineSize ---> 0.0
Type vs  Horsepower ---> 0.0
Type vs  RPM ---> 0.0
Type vs  Revpermile ---> 0.0
Type vs  Fueltankcapacity ---> 0.0
Type vs  Passengers ---> 0.0
Type vs  Length ---> 0.0
Type vs  Wheelbase ---> 0.0
Type vs  Width ---> 0.0
Type vs  Turncircle ---> 0.0
Type vs  Rearseatroom ---> 0.0
Type vs  Luggageroom ---> 0.0
Type vs  Weight ---> 0.0


# Chi-Squre test

In [12]:
def chisqure(df,cat1,cat2):
    from scipy.stats import chi2_contingency
    ct = pd.crosstab(df[cat1],df[cat2])
    a,b,c,d = chi2_contingency(ct)
    return round(b,4)

In [13]:
for i in X.columns:
    if(X[i].dtypes=="object"):
        x = chisqure(A,"Type",i)
        if(x<0.05):
            imp_col.append(i)
            print("Type vs ",i,"--->",x)

Type vs  AirBags ---> 0.0002
Type vs  DriveTrain ---> 0.0001
Type vs  Cylinders ---> 0.0
Type vs  Mantransavail ---> 0.0
Type vs  Origin ---> 0.0151


In [14]:
imp_col

['MinPrice',
 'Price',
 'MaxPrice',
 'MPGcity',
 'MPGhighway',
 'EngineSize',
 'Horsepower',
 'RPM',
 'Revpermile',
 'Fueltankcapacity',
 'Passengers',
 'Length',
 'Wheelbase',
 'Width',
 'Turncircle',
 'Rearseatroom',
 'Luggageroom',
 'Weight',
 'AirBags',
 'DriveTrain',
 'Cylinders',
 'Mantransavail',
 'Origin']

In [15]:
X=X[imp_col]

In [16]:
X.head()

,MinPrice,Price,MaxPrice,MPGcity,MPGhighway,EngineSize,Horsepower,RPM,Revpermile,Fueltankcapacity,...,Width,Turncircle,Rearseatroom,Luggageroom,Weight,AirBags,DriveTrain,Cylinders,Mantransavail,Origin
0,12.9,15.9,18.8,25,31,1.8,140,6300,2890,13.2,...,68,37,26.5,11.0,2705,None,Front,4,Yes,non-USA
1,29.2,33.9,38.7,18,25,3.2,200,5500,2335,18.0,...,71,38,30.0,15.0,3560,Driver & Passenger,Front,6,Yes,non-USA
2,25.9,29.1,32.3,20,26,2.8,172,5500,2280,16.9,...,67,37,28.0,14.0,3375,Driver only,Front,6,Yes,non-USA
3,30.8,37.7,44.6,19,26,2.8,172,5500,2535,21.1,...,70,37,31.0,17.0,3405,Driver only,Front,6,Yes,non-USA
4,23.7,30.0,36.2,22,30,3.5,208,5700,2545,21.1,...,69,39,27.0,13.0,3640,Driver only,Rear,4,Yes,non-USA


# Data Preprocessing

In [17]:
con = []
cat = []
for i in X.columns:
    if(X[i].dtypes=="object"):
        cat.append(i)
    else:
        con.append(i)
        
X1 = pd.get_dummies(X[cat])                           # one hot encoding for caategorical features
from sklearn.preprocessing import StandardScaler      # standard scaler for continous features
ss = StandardScaler()
X2 = pd.DataFrame(ss.fit_transform(X[con]),columns=con)

In [18]:
Xnew = X2.join(X1)

In [19]:
Xnew.head()

,MinPrice,Price,MaxPrice,MPGcity,MPGhighway,EngineSize,Horsepower,RPM,Revpermile,Fueltankcapacity,...,Cylinders_3,Cylinders_4,Cylinders_5,Cylinders_6,Cylinders_8,Cylinders_rotary,Mantransavail_No,Mantransavail_Yes,Origin_USA,Origin_non-USA
0,-0.485787,-0.375720,-0.282465,0.471312,0.360925,-0.841022,-0.073484,1.717489,1.129530,-1.062184,...,0,1,0,0,0,0,0,1,0,1
1,1.388017,1.497844,1.531409,-0.781032,-0.770514,0.515869,1.078322,0.369586,0.005661,0.409445,...,0,0,0,1,0,0,0,1,0,1
2,1.008658,0.998227,0.948052,-0.423219,-0.581941,0.128186,0.540813,0.369586,-0.105713,0.072197,...,0,0,0,1,0,0,0,1,0,1
3,1.571949,1.893374,2.069191,-0.602126,-0.581941,0.128186,0.540813,0.369586,0.410659,1.359872,...,0,0,0,1,0,0,0,1,0,1
4,0.755752,1.091905,1.303535,-0.065407,0.172352,0.806631,1.231897,0.706562,0.430909,1.359872,...,0,1,0,0,0,0,0,1,0,1


# Data Divide into training and testing set

In [20]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(Xnew,Y,test_size=0.2,random_state=21)

# Create Model 

In [21]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
model = lr.fit(xtrain,ytrain)
pred_tr = model.predict(xtrain)
pred_ts = model.predict(xtest)

# Find training and testing accuracy score

In [22]:
from sklearn.metrics import accuracy_score,confusion_matrix
tr_acc = accuracy_score(ytrain,pred_tr)
td_acc = accuracy_score(ytest,pred_ts)

In [23]:
tr_acc

1.0

In [24]:
td_acc

0.9473684210526315

In [25]:
confusion_matrix(ytest,pred_ts)

array([[1, 0, 0, 0, 0, 0],
       [0, 3, 0, 0, 0, 0],
       [0, 0, 4, 0, 0, 0],
       [1, 0, 0, 3, 0, 0],
       [0, 0, 0, 0, 5, 0],
       [0, 0, 0, 0, 0, 2]], dtype=int64)

In [26]:
ytest["testpred"] = pred_ts

In [27]:
ytest

,Type,testpred
23,Small,Compact
86,Van,Van
91,Compact,Compact
21,Large,Large
17,Large,Large
82,Small,Small
34,Sporty,Sporty
27,Sporty,Sporty
10,Midsize,Midsize
40,Sporty,Sporty
